In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_age_gp = pd.read_csv("zip_code_age_gp.csv", index_col="Unnamed: 0")

In [3]:
df_age_gp.head(2)

,female[0-5],male[0-5],female[6-11],male[6-11],female[12-17],male[12-17],female[18-23],male[18-23],female[24-29],male[24-29],...,female[42-47],male[42-47],female[48-53],male[48-53],female[54-59],male[54-59],female[60-65],male[60-65],female[66-102],male[66-102]
10026,1242,1141,1124,1201,1194,1171,1514,1425,2134,1773,...,1676,1609,1528,1432,1201,1071,911,699,1860,1071
10027,1904,1828,1707,1739,1896,1929,5573,4126,4402,3858,...,2388,2263,2184,1971,1939,1661,1476,1243,3329,1930


In [4]:
df_wealthy = pd.read_csv("../data-collect/wealth_data.csv", sep="\t")
df_wealthy = df_wealthy.dropna(axis="index", how='any')
df_wealthy["zip_code"] = df_wealthy["zip_code"].astype('int64')
df_wealthy["count"] = df_wealthy["count"].replace("**", 0).astype('int64')
df_wealthy = df_wealthy[df_wealthy["zip_code"].isin(df_age_gp.index)]

In [10]:
for i, row in df_wealthy.iterrows():
    lb, ub = row["income_range"].replace(",", "").split("-")
    if ub != "...":
        lb, ub = int(lb), int(ub)
    else:
        lb, ub = int(lb), int(lb)
    avg = (lb+ub)//2
    df_wealthy.loc[i, "avg_income"] = avg
    
df_wealthy["avg_income"] = df_wealthy["avg_income"].astype('int64')
df_wealthy.head(6)

,zip_code,income_range,count,avg_income
1,10001,"1-25,000",3760,12500
2,10001,"25,000-50,000",2430,37500
3,10001,"50,000-75,000",1930,62500
4,10001,"75,000-100,000",1340,87500
5,10001,"100,000-200,000",2480,150000
6,10001,"200,000-...",2370,200000


In [8]:
def my_groupby_agg():
    for zc in df_age_gp.index:
        rows = df_wealthy.zip_code==zc
        yield {"zip_code": zc,
               "weight_income": (df_wealthy[rows]["count"]@df_wealthy[rows]["avg_income"])/np.sum(df_wealthy[rows]["count"])}
        
# df_weight_income = pd.DataFrame(my_groupby_agg())[["zip_code", "weight_income"]]

In [9]:
pd.merge(pd.DataFrame(my_groupby_agg())[["zip_code", "weight_income"]],
         df_age_gp, right_index=True, left_on="zip_code").head()

/anaconda3/envs/SP/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


,zip_code,weight_income,female[0-5],male[0-5],female[6-11],male[6-11],female[12-17],male[12-17],female[18-23],male[18-23],...,female[42-47],male[42-47],female[48-53],male[48-53],female[54-59],male[54-59],female[60-65],male[60-65],female[66-102],male[66-102]
0,10026,54714.673913,1242,1141,1124,1201,1194,1171,1514,1425,...,1676,1609,1528,1432,1201,1071,911,699,1860,1071
1,10027,50295.347146,1904,1828,1707,1739,1896,1929,5573,4126,...,2388,2263,2184,1971,1939,1661,1476,1243,3329,1930
2,10030,38531.860970,1033,1058,951,982,1150,1165,1412,1258,...,1352,1136,1243,1078,924,833,703,527,1412,938
3,10037,48464.912281,548,504,471,529,590,610,741,664,...,867,647,829,603,782,468,693,449,1916,910
4,10039,38751.840943,968,1035,1006,1030,1086,1074,1195,1151,...,1154,977,1079,838,838,578,737,415,1727,839
